
# Emotion Classification — MobileNet (TensorFlow/Keras)

This notebook trains a MobileNet-based image classifier using `flow_from_directory`.
It follows a clean, **cell-by-cell** structure so you can adapt paths and parameters easily.


In [ ]:

# %% [markdown]
# ## 1. Environment & Version Check
import sys, tensorflow as tf, numpy as np, PIL

print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
print("Pillow:", PIL.__version__)


In [ ]:

# %% [markdown]
# ## 2. Configuration
# Update these paths to your dataset.
# The directory structure should be:
# train_dir/
#   class_a/
#     img1.jpg, img2.jpg, ...
#   class_b/
#     ...
# val_dir/
#   class_a/
#   class_b/

from pathlib import Path

# >>>> EDIT THESE <<<<
train_dir = Path("/path/to/fer2013/train")       # e.g., D:/data/fer2013/train
val_dir   = Path("/path/to/fer2013/validation")  # e.g., D:/data/fer2013/validation

# Image size for MobileNet-like backbones
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
SEED = 42

# Choose backbone
BACKBONE = "MobileNetV2"   # options: "MobileNet", "MobileNetV2"
FREEZE_UP_TO = 0           # number of layers to freeze (0 = train all)
LEARNING_RATE = 1e-3

# Output files
OUT_DIR = Path("./outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_BEST_PATH = str(OUT_DIR / "emotion_mobilenet_best.h5")
MODEL_LAST_PATH = str(OUT_DIR / "emotion_mobilenet_last.h5")
HISTORY_PATH    = str(OUT_DIR / "train_history.npy")


In [ ]:

# %% [markdown]
# ## 3. Data Loading (ImageDataGenerator / flow_from_directory)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

assert train_dir.exists(), f"Train directory not found: {train_dir}"
assert val_dir.exists(), f"Validation directory not found: {val_dir}"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    str(train_dir),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

val_gen = val_datagen.flow_from_directory(
    str(val_dir),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_classes = len(train_gen.class_indices)
print("Detected classes:", train_gen.class_indices)
print("num_classes =", num_classes)


In [ ]:

# %% [markdown]
# ## 4. Build Model (MobileNet / MobileNetV2 + Custom Head)
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet, MobileNetV2

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

if BACKBONE == "MobileNet":
    base = MobileNet(weights='imagenet', include_top=False, input_tensor=inputs)
elif BACKBONE == "MobileNetV2":
    base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
else:
    raise ValueError("Unsupported BACKBONE. Use 'MobileNet' or 'MobileNetV2'.")

# Freeze first N layers if requested
for i, layer in enumerate(base.layers):
    layer.trainable = i >= FREEZE_UP_TO

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs=inputs, outputs=outputs)
model.summary()


In [ ]:

# %% [markdown]
# ## 5. Compile
from tensorflow.keras.optimizers import Adam

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy']
)


In [ ]:

# %% [markdown]
# ## 6. Callbacks
import tensorflow as tf

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    MODEL_BEST_PATH,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

callbacks = [checkpoint_cb, earlystop_cb, reduce_lr_cb]


In [ ]:

# %% [markdown]
# ## 7. Train
import math

steps_per_epoch = math.ceil(train_gen.samples / BATCH_SIZE)
validation_steps = math.ceil(val_gen.samples / BATCH_SIZE)

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    callbacks=callbacks
)

# Save last epoch model & history
model.save(MODEL_LAST_PATH)
import numpy as np
np.save(HISTORY_PATH, history.history, allow_pickle=True)
print("Saved:", MODEL_LAST_PATH, "and training history:", HISTORY_PATH)


In [ ]:

# %% [markdown]
# ## 8. Plot Training Curves
import matplotlib.pyplot as plt
import numpy as np

hist = np.load(HISTORY_PATH, allow_pickle=True).item()

plt.figure()
plt.plot(hist['loss'], label='train_loss')
plt.plot(hist['val_loss'], label='val_loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()
plt.plot(hist['accuracy'], label='train_acc')
plt.plot(hist['val_accuracy'], label='val_acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:

# %% [markdown]
# ## 9. Evaluation & Sample Predictions
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate
val_loss, val_acc = model.evaluate(val_gen, steps=validation_steps, verbose=1)
print(f"Validation — loss: {val_loss:.4f}, acc: {val_acc:.4f}")

# Predictions
val_gen.reset()
pred_probs = model.predict(val_gen, steps=validation_steps, verbose=1)
y_pred = np.argmax(pred_probs, axis=1)
y_true = val_gen.classes

# Align lengths (in case of last partial batch)
y_true = y_true[:len(y_pred)]

target_names = list(val_gen.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:

# %% [markdown]
# ## 10. Confusion Matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
fig = plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(target_names))
plt.xticks(tick_marks, target_names, rotation=45, ha='right')
plt.yticks(tick_marks, target_names)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


In [ ]:

# %% [markdown]
# ## 11. Single-Image Inference Helper
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from pathlib import Path

def predict_image(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    preds = model.predict(arr)
    idx = int(np.argmax(preds))
    classes = list(train_gen.class_indices.keys())
    return classes[idx], float(np.max(preds))

# Example:
# test_img = Path("/path/to/an/image.jpg")
# label, prob = predict_image(test_img)
# print(label, prob)


In [ ]:

# %% [markdown]
# ## 12. (Optional) Fine-Tuning: Unfreeze More Layers and Train with Lower LR
# Run this after initial training if you want to fine-tune deeper layers.

# Example: unfreeze last N layers
N_UNFREEZE = 30  # e.g., unfreeze last 30 layers
for layer in model.layers[-N_UNFREEZE:]:
    layer.trainable = True

# Re-compile with lower LR
from tensorflow.keras.optimizers import Adam
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

history_ft = model.fit(
    train_gen,
    epochs=5,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    callbacks=callbacks
)
